In [2]:
!pip install -q langchain langgraph langchain-groq python-dotenv \
                langchain-community faiss-cpu sentence-transformers requests

In [5]:
import os
from dotenv import load_dotenv

load_dotenv()
assert os.getenv("GROQ_API_KEY") is not None, "❌ GROQ_API_KEY not loaded"
print("✅ GROQ API Key Loaded")

✅ GROQ API Key Loaded


In [4]:
from typing import Annotated, TypedDict, Literal
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver
from langchain_groq import ChatGroq
from langchain_core.tools import tool

In [5]:
llm_memory = ChatGroq(
    model="llama-3.1-8b-instant",
    api_key=os.getenv("GROQ_API_KEY")
)

In [6]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [7]:
def memory_chatbot(state: State):
    return {"messages": [llm_memory.invoke(state["messages"])]}

In [8]:
builder = StateGraph(State)
builder.add_node("chatbot", memory_chatbot)
builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)

memory = MemorySaver()
memory_agent = builder.compile(checkpointer=memory)

print("✅ Memory Agent Ready")

✅ Memory Agent Ready


In [9]:
config = {"configurable": {"thread_id": "session_1"}}

memory_agent.invoke(
    {"messages": [("user", "My name is Sid.")]},
    config=config
)

result = memory_agent.invoke(
    {"messages": [("user", "What is my name?")]},
    config=config
)

print(result["messages"][-1].content)

Your name is Sid.


In [10]:
import requests

@tool
def get_weather(city: str):
    """Get current temperature of a city."""
    geo = requests.get(
        "https://geocoding-api.open-meteo.com/v1/search",
        params={"name": city, "count": 1, "format": "json"}
    ).json()

    if not geo.get("results"):
        return f"City {city} not found."

    lat = geo["results"][0]["latitude"]
    lon = geo["results"][0]["longitude"]

    weather = requests.get(
        "https://api.open-meteo.com/v1/forecast",
        params={"latitude": lat, "longitude": lon, "current": "temperature_2m"}
    ).json()

    return f"The temperature in {city} is {weather['current']['temperature_2m']}°C"

In [11]:
llm_weather = ChatGroq(
    model="llama-3.1-8b-instant",
    api_key=os.getenv("GROQ_API_KEY")
)

llm_weather_tools = llm_weather.bind_tools([get_weather])

In [12]:
class WeatherState(TypedDict):
    messages: Annotated[list, add_messages]

def weather_chatbot(state: WeatherState):
    return {"messages": [llm_weather_tools.invoke(state["messages"])]}

tool_node = ToolNode(tools=[get_weather])

def weather_router(state: WeatherState) -> Literal["tools", END]:
    if state["messages"][-1].tool_calls:
        return "tools"
    return END

builder = StateGraph(WeatherState)
builder.add_node("chatbot", weather_chatbot)
builder.add_node("tools", tool_node)

builder.add_edge(START, "chatbot")
builder.add_conditional_edges("chatbot", weather_router, {"tools": "tools", END: END})
builder.add_edge("tools", "chatbot")

weather_agent = builder.compile()
print("✅ Weather Agent Ready")

✅ Weather Agent Ready


In [13]:
weather_agent.invoke(
    {"messages": [("user", "What is the weather in London?")]}
)

{'messages': [HumanMessage(content='What is the weather in London?', additional_kwargs={}, response_metadata={}, id='ff7542f8-bc09-4ba0-b408-36b13e2aab51'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 's5x7fj79r', 'function': {'arguments': '{"city":"London"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 219, 'total_tokens': 233, 'completion_time': 0.025959582, 'completion_tokens_details': None, 'prompt_time': 0.015004666, 'prompt_tokens_details': None, 'queue_time': 0.055670464, 'total_time': 0.040964248}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b8a74-77d3-75f2-9faa-34a2611b8ae2-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'London'}, 'id': 's5x7fj79r', 'type': 'tool_call'}], usage_metadata={'input_tokens': 219, 'output_t

In [14]:
import random
from langchain_core.tools import tool

@tool
def flip_coin():
    """Flip a coin and return Heads or Tails."""
    return random.choice(["Heads", "Tails"])

@tool
def roll_dice():
    """Roll a six-sided dice and return a number between 1 and 6."""
    return random.randint(1, 6)

In [15]:
llm_game = ChatGroq(
    model="llama-3.1-8b-instant",
    api_key=os.getenv("GROQ_API_KEY")
)

llm_game_tools = llm_game.bind_tools([flip_coin, roll_dice])

In [16]:
class GameState(TypedDict):
    messages: Annotated[list, add_messages]

def game_chatbot(state: GameState):
    return {"messages": [llm_game_tools.invoke(state["messages"])]}

tool_node = ToolNode(tools=[flip_coin, roll_dice])

def game_router(state: GameState):
    if state["messages"][-1].tool_calls:
        return "tools"
    return END

builder = StateGraph(GameState)
builder.add_node("chatbot", game_chatbot)
builder.add_node("tools", tool_node)

builder.add_edge(START, "chatbot")
builder.add_conditional_edges("chatbot", game_router, {"tools": "tools", END: END})
builder.add_edge("tools", "chatbot")

game_agent = builder.compile()
print("🎲 Game Agent Ready")

🎲 Game Agent Ready


In [17]:
game_agent.invoke(
    {"messages": [("user", "Flip a coin and roll a dice")]}
)

{'messages': [HumanMessage(content='Flip a coin and roll a dice', additional_kwargs={}, response_metadata={}, id='a9c0588f-5c51-4e5a-8430-86dd93d8fe21'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'wwk2d8xkw', 'function': {'arguments': '{}', 'name': 'flip_coin'}, 'type': 'function'}, {'id': 'wh5cv4z60', 'function': {'arguments': '{}', 'name': 'roll_dice'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 276, 'total_tokens': 295, 'completion_time': 0.023416283, 'completion_tokens_details': None, 'prompt_time': 0.015859737, 'prompt_tokens_details': None, 'queue_time': 0.049505613, 'total_time': 0.03927602}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b8a74-7f3f-7be2-b899-845a18c149e0-0', tool_calls=[{'name': 'flip_coin', 'args': {}, 'id': 'wwk2d8xkw', 'type': 'tool

In [18]:
!pip install -q langchain-community

In [2]:
import sys
!{sys.executable} -m pip install sentence-transformers langchain-huggingface

  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached safetensors-0.7.0-cp38-abi3-macosx_11_0_arm64.whl.metadata (4.1 kB)
  Using cached fsspec-2025.12.0-py3-none-any.whl.metadata (10 kB)
  Using cached hf_xet-1.2.0-cp37-abi3-macosx_11_0_arm64.whl.metadata (4.9 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.6.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
  Using cached markupsafe-3.0.3-cp311-cp311-macosx_11_0_arm64.whl.metadata (2.7 kB)
  Using cached joblib-1.5.3-py3-none-any.whl.metadata (5.5 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached transf

In [2]:
import sys
!{sys.executable} -m pip install faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 5.3 MB/s  0:00:005.7 MB/s eta 0:00:01


In [1]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document

embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2"
)

docs = [
    Document(page_content="Lunch is served at 12:30 PM in the canteen."),
    Document(page_content="The WiFi password is Grapes123."),
    Document(page_content="IT desk is on the 2nd floor.")
]

vector_db = FAISS.from_documents(docs, embeddings)
print("✅ Vector DB Ready")

/opt/anaconda3/envs/groq-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Vector DB Ready


In [3]:
from langchain_core.tools import tool

@tool
def search_knowledge_base(query: str):
    """
    Search the internal knowledge base for company information
    like lunch, wifi, or IT support.
    """
    results = vector_db.similarity_search(query, k=1)
    
    if results:
        return results[0].page_content
    
    return "No relevant information found."

In [6]:
from langchain_groq import ChatGroq
import os

llm_rag = ChatGroq(
    model="llama-3.1-8b-instant",
    api_key=os.getenv("GROQ_API_KEY")
)

llm_rag_tools = llm_rag.bind_tools([search_knowledge_base])

In [7]:
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages

class RAGState(TypedDict):
    messages: Annotated[list, add_messages]

In [8]:
def rag_chatbot(state: RAGState):
    return {"messages": [llm_rag_tools.invoke(state["messages"])]}

In [9]:
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, START, END

tool_node = ToolNode(tools=[search_knowledge_base])

def rag_router(state: RAGState):
    if state["messages"][-1].tool_calls:
        return "tools"
    return END

In [10]:
builder = StateGraph(RAGState)

builder.add_node("chatbot", rag_chatbot)
builder.add_node("tools", tool_node)

builder.add_edge(START, "chatbot")
builder.add_conditional_edges(
    "chatbot",
    rag_router,
    {"tools": "tools", END: END}
)
builder.add_edge("tools", "chatbot")

rag_agent = builder.compile()
print("📚 RAG Agent Ready")

📚 RAG Agent Ready


In [11]:
response = rag_agent.invoke(
    {"messages": [("user", "Where can I get lunch?")]}
)

print(response["messages"][-1].content)

You can reach the IT support team at it-support@company.com or 555-555-5555.
